<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-необходимых-библиотек" data-toc-modified-id="Импорт-необходимых-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт необходимых библиотек</a></span></li><li><span><a href="#Открытие-файла" data-toc-modified-id="Открытие-файла-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Открытие файла</a></span></li><li><span><a href="#Вывод-первых-десяти-строк-таблицы" data-toc-modified-id="Вывод-первых-десяти-строк-таблицы-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Вывод первых десяти строк таблицы</a></span></li><li><span><a href="#Вывод-основной-информации-о-датафрейме" data-toc-modified-id="Вывод-основной-информации-о-датафрейме-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Вывод основной информации о датафрейме</a></span></li><li><span><a href="#Преобразование-текста" data-toc-modified-id="Преобразование-текста-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Преобразование текста</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Разбиение-данных-на-целевой-признак-и-признаки" data-toc-modified-id="Разбиение-данных-на-целевой-признак-и-признаки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Разбиение данных на целевой признак и признаки</a></span></li><li><span><a href="#Разбиение-данных-на-выборки" data-toc-modified-id="Разбиение-данных-на-выборки-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Разбиение данных на выборки</a></span></li><li><span><a href="#Пайплайн" data-toc-modified-id="Пайплайн-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Пайплайн</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

## Подготовка

### Импорт необходимых библиотек

In [186]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords as nltk_stopwords
stoplist = set(nltk_stopwords.words('english')) 
from sklearn.feature_extraction.text import TfidfVectorizer
import re

from sklearn.model_selection import train_test_split 

from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

### Открытие файла

In [187]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv', engine='python')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', engine='python')

### Вывод первых десяти строк таблицы

In [188]:
data.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


### Вывод основной информации о датафрейме

In [189]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


### Преобразование текста

Очистим текст от лишних символов с помощью регулярных выражений:

In [190]:
data['text'] = data['text'].str.replace(r'[^\'a-zA-Z ]', ' ', regex=True)

Лемматизируем текст:

In [191]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

data['text_lemmatized'] = data.text.apply(lemmatize_text) 

In [192]:
data['text_lemmatized'] = [item.lower() for item in data['text_lemmatized']]

### Вывод

Данные были преобразованы с помощью регулярных выражений и лемматизации. Необходимо их разбить на целевой признак и признаки — столбец `toxic` и столбец `text` соответственно.

## Обучение

### Разбиение данных на целевой признак и признаки

In [193]:
features = data['text_lemmatized']
target = data['toxic']

### Разбиение данных на выборки

Разобьем данные на обучающую и тестовую выборку в отношении 3:1:

In [194]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

### Пайплайн

In [195]:
def search_params(pipeline, params):
    search = RandomizedSearchCV(pipeline, param_distributions=params, scoring='f1')
    search.fit(features_train, target_train)
    
    return search.best_estimator_, search.best_score_

### Логистическая регрессия

In [198]:
pipe_params = Pipeline([
    ('vect', TfidfVectorizer(stop_words=stoplist)),
    ('model', LogisticRegression(max_iter=1000, random_state=12345))
])

params = {'model__C' : np.linspace(5, 12, 5)}

best_reg, reg_f1 = search_params(pipe_params, params)
print(best_reg, reg_f1)

C:\Users\sushi\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('model',
                 LogisticRegression(C=12.0, max_iter=1000,
                                    random_state=12345))]) 0.7672258938077886


### Дерево решений

In [204]:
pipe_params = Pipeline([
    ('vect', TfidfVectorizer(stop_words=stoplist)),
    ('model', DecisionTreeClassifier(random_state=12345))
])

params = {'model__max_depth' : range(5, 15, 2)}

best_forest, forest_f1 = search_params(pipe_params, params)
print(best_forest, forest_f1)

C:\Users\sushi\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('model',
                 DecisionTreeClassifier(max_depth=13, random_state=12345))]) 0.6073238834969654


### Вывод

С помощью `Pipeline` и `RandomizedSearchCV` были подобраны такие параметры для моделей случайного леса и логистической регрессии, чтобы метрика F1 была наибольшей. Наилучшую метрику F1 на валидационной выборке показала модель логистической регрессии — около 0.77. 

## Выводы

   Проверим метрику F1 модели логистической регрессии на тестовой выборке:

In [201]:
print(f1_score(best_reg.predict(features_test), target_test))

0.778393351800554


Метрика показала больше требуемых 0.75. 

Таким образом, данные были предобработаны с помощью регулярных выражений и лемматизации, после чего они были разбиты на тренировочную и тестовую выборки. Поиск гиперпараметров был совершен через `Pipeline` и `RandomizedSearchCV`. Наилучшей моделью, судя по метрике F1, оказалась модель логистической регрессии: на валидационной выборке она составила 0.76, а на валидационной — около 0.77.